## CAO Points Analysis

https://www.cao.ie/index.php?page=points&p=2021&bb=points

***

In [1]:
# To access a URL from python
# import urllib

# Convenient HTTP requests
import requests as rq

# Regular expressions
import re

# Should always call this out on the requirements txt file as it doesn't come as standard package with python
# Only if you have anaconda does it come with the package

# Dates and times
import datetime as dt

In [2]:
# Fetch the CAO points URL.
resp = rq.get('http://www2.cao.ie/points/l8.php')
# Have a quick peek (http code for ok)
resp
# To see the whole text
# resp.text

<Response [200]>

### Save Original data set

***

In [3]:
# Get the current date and time
now = dt.datetime.now()

# Format as a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [4]:
# Create a file path for the original data
path = 'data/cao2021_' + nowstr + 'html'

<br>

### Error on server

***

Technically, the server says we should decode as per:
```
Content-Type: text/html; charset=iso-8859-1
```
However, one line uses \x96 which isn't defined in iso-8859-1.
<br>Therefore we use the similar decoding standard cp1252, which is very similar but includes #x96.

In [5]:
# The server uses the wrong encoding, fix it
original_encoding = resp.encoding
# Change to cp1252
resp.encoding = 'cp1252'

In [6]:
# Save the original html file
with open (path, 'w') as f:
    f.write(resp.text)

<br>

### Use regular expressions to select lines we want

***

In [7]:
# Compile the regular expression for matching lines
re_course = re.compile('([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')

<br>

### Loop through the lines of the response

***

In [9]:
# The file path for the csv file
path = 'data/cao2021_csv_' + nowstr + '.csv'

# Keep track of how many courses we process 
no_lines = 0

# Open the cav file for writing
with open (path, 'w') as f:
    # Loop through lines of the respone
    for line in resp.iter_lines():
        # Decode the line, changed from ISO-8859-1 to cp1252
        # as one line uses \x96 which isn't defined in iso-8859-1
        dline = line.decode('cp1252')
        # Match only the lines representing courses
        if re_course.fullmatch(dline):
            # Add one to the lines counter
            no_lines = no_lines + 1
            # Uncomment next lines to see the original
            # print(line)
            # Pick out the relevant parts of the matched line
            # csv_version = re_course.sub(r'\1,\2,\3,\4', dline)
            # Print the CSV-style line
            # print(csv_version)
            # Split the line on two or more spaces
            linesplit = re.split('  +', dline)
            # print(linesplit)
            # print(','.join(linesplit))
            # Rejoin the substrings with commas between them
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines
print(f"Total number of lines is {no_lines}.")

Total number of lines is 922.


### End